<a href="https://colab.research.google.com/github/beyzaturku/image_processing_in_autonomous_vehicles/blob/main/lane_and_car_detection_tracking_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q opencv-python
!pip install -q ultralytics
!pip install -q deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 69.2 MB/s eta 0:00:00


In [2]:
import os
import cv2
import numpy as np
import torch
import time
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
from google.colab.patches import cv2_imshow
import moviepy.editor as editor
from IPython.display import display, HTML
import matplotlib.pyplot as plt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


  if event.key is 'enter':



In [3]:
print("Lütfen bir test videosu yükleyin:")
uploaded = files.upload()
test_video = list(uploaded.keys())[0]

Lütfen bir test videosu yükleyin:


Saving project_video.mp4 to project_video.mp4


In [4]:
os.makedirs("output", exist_ok=True)

#### **ŞERİT TAKİP ALGORİTMASI**

In [5]:
# Renk eşikleme
def color_threshold(img):
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)

    white = cv2.inRange(hls[:, :, 1], 200, 255)
    yellow = cv2.inRange(hls[:, :, 0], 15, 35) & cv2.inRange(hls[:, :, 2], 100, 255)
    light = cv2.inRange(lab[:, :, 0], 190, 255)
    grayish = cv2.inRange(hls, (0, 30, 60), (180, 80, 200))

    return cv2.bitwise_or(cv2.bitwise_or(cv2.bitwise_or(white, yellow), light), grayish)

In [6]:
# Kenar tespiti
def edge_detection(gray):
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges

In [7]:
# ROI belirleme
def region_of_interest(img):
    h, w = img.shape[:2]
    mask = np.zeros_like(img)
    polygon = np.array([[
        (int(w * 0.1), h),
        (int(w * 0.4), int(h * 0.6)),
        (int(w * 0.6), int(h * 0.6)),
        (int(w * 0.9), h)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(img, mask)

In [8]:
# Hough çizgileri
def hough_lines(img):
    lines = cv2.HoughLinesP(img, 1, np.pi/180, 50, minLineLength=40, maxLineGap=100)
    if lines is None:
        return []
    filtered = []
    for x1, y1, x2, y2 in lines.reshape(-1, 4):
        if x2 == x1: continue
        slope = (y2 - y1) / (x2 - x1)
        angle = abs(np.degrees(np.arctan(slope)))
        if 20 < angle < 80:
            filtered.append((x1, y1, x2, y2, slope))
    return filtered

In [9]:
# Şerit çizgileri uydurma ve merkez hesaplama
def fit_lane_lines(img, lines):
    h, w = img.shape[:2]
    left_points, right_points = [], []
    left_slope, right_slope = None, None

    for x1, y1, x2, y2, slope in lines:
        if slope < 0:
            left_points += [(x1, y1), (x2, y2)]
        else:
            right_points += [(x1, y1), (x2, y2)]

    def make_line(points):
        if not points:
            return None, None
        x, y = zip(*points)
        poly = np.polyfit(y, x, 1)

        y1 = int(h)
        y2 = int(h * 0.75)

        x1 = int(np.polyval(poly, y1))
        x2 = int(np.polyval(poly, y2))

        if not (0 <= x1 < w and 0 <= x2 < w):
            return None, None

        slope = -poly[0]
        return (x1, y1, x2, y2), slope

    left_line, left_slope = make_line(left_points)
    right_line, right_slope = make_line(right_points)

    lane_img = np.zeros_like(img)
    lane_center = None
    car_offset = None

    if left_line and right_line:
        cv2.line(lane_img, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (0, 255, 0), 8)
        cv2.line(lane_img, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (0, 0, 255), 8)

        left_base = left_line[0]
        right_base = right_line[0]
        lane_center = (left_base + right_base) // 2
        car_center = w // 2
        car_offset = (car_center - lane_center) * 3.7 / 700

        cv2.putText(lane_img, f"Slope L: {left_slope:.2f}", (left_line[0], left_line[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        cv2.putText(lane_img, f"Slope R: {right_slope:.2f}", (right_line[0], right_line[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    return lane_img, car_offset

### **ARAÇ TESPİT VE TAKİP ALGORİTMASI**

In [10]:
# YOLO modelini yükleme
print("YOLOv8 modeli yükleniyor...")
model = YOLO("yolov8m.pt")

YOLOv8 modeli yükleniyor...


100%|██████████| 49.7M/49.7M [00:00<00:00, 252MB/s]


In [11]:
# Araç sınıfları tanımlama
arac_siniflari = [2, 3, 5, 7]  # 2:araba, 3:motosiklet, 5:otobüs, 7:kamyon
sinif_isimleri = {2: "Araba", 3: "Motosiklet", 5: "Otobus", 7: "Kamyon"}

In [12]:
# DeepSORT Takipçisini Başlatma
tracker = DeepSort(max_age=30)

# Araç renklerini tutarlı hale getirmek için seed ayarı
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(100, 3), dtype="uint8")

### **BİRLEŞTİRİLMİŞ PIPELINE FONKSİYONU**

In [13]:
def process_integrated_video(input_path, output_path, save_frames=True):
    cap = cv2.VideoCapture(input_path)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    arac_sayilari = {2: 0, 3: 0, 5: 0, 7: 0}
    aktif_araclar = {2: 0, 3: 0, 5: 0, 7: 0}

    track_history = {}
    track_speed = {}

    last_time = time.time()
    prev_time = cv2.getTickCount()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()
        frame_count += 1

        original_frame = frame.copy()
        debug_img = frame.copy()

        # ŞERİT TESPİT İŞLEMLERİ
        color_mask = color_threshold(frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edge_mask = edge_detection(gray)

        combined = cv2.bitwise_or(color_mask, edge_mask)
        roi = region_of_interest(combined)

        lines = hough_lines(roi)
        lane_img, car_offset = fit_lane_lines(frame, lines)

        lane_overlay = cv2.addWeighted(frame, 0.8, lane_img, 1, 0)

        # ARAÇ TESPİT VE TAKİP İŞLEMLERİ

        curr_time = cv2.getTickCount()
        time_diff = (curr_time - prev_time) / cv2.getTickFrequency()
        fps_calc = 1.0 / time_diff
        prev_time = curr_time

        results = model(lane_overlay)

        detections = []

        for result in results:
            boxes = result.boxes.cpu().numpy()

            for box in boxes:
                class_id = int(box.cls[0])
                if class_id in arac_siniflari:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = float(box.conf[0])

                    if confidence > 0.5:
                        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                        box_width = int((x2 - x1) * 0.8)
                        box_height = int((y2 - y1) * 0.8)

                        x1_new = max(0, center_x - box_width // 2)
                        y1_new = max(0, center_y - box_height // 2)
                        x2_new = min(width, center_x + box_width // 2)
                        y2_new = min(height, center_y + box_height // 2)

                        detection = [[x1_new, y1_new, x2_new - x1_new, y2_new - y1_new], confidence, class_id]
                        detections.append(detection)

        # DeepSORT ile araç takibi
        if detections:
            tracks = tracker.update_tracks(detections, frame=lane_overlay)

            for cls_id in aktif_araclar:
                aktif_araclar[cls_id] = 0

            for track in tracks:
                if not track.is_confirmed():
                    continue

                ltrb = track.to_ltrb()
                x1, y1, x2, y2 = map(int, ltrb)
                track_id = track.track_id

                if isinstance(track_id, str):
                    track_id = int(track_id)

                class_id = None
                if hasattr(track, 'det_class'):
                    class_id = track.det_class
                elif hasattr(track, 'class_id'):
                    class_id = track.class_id

                color = [int(c) for c in COLORS[track_id % len(COLORS)]]

                cv2.rectangle(lane_overlay, (x1, y1), (x2, y2), color, 2)

                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                center_point = (center_x, center_y)

                if track_id not in track_history:
                    track_history[track_id] = []

                track_history[track_id].append(center_point)
                if len(track_history[track_id]) > 20:
                    track_history[track_id].pop(0)

                if len(track_history[track_id]) > 1:
                    for i in range(1, len(track_history[track_id])):
                        pt1 = track_history[track_id][i-1]
                        pt2 = track_history[track_id][i]
                        cv2.line(lane_overlay, pt1, pt2, color, 2)

                if track_id in track_speed:
                    last_pos, last_time = track_speed[track_id]

                    dist = np.sqrt((center_x - last_pos[0])**2 + (center_y - last_pos[1])**2)
                    time_elapsed = frame_count - last_time

                    speed = dist / time_elapsed * fps * 0.1

                    cv2.putText(lane_overlay, f"{speed:.1f} km/s", (x1, y1 - 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                track_speed[track_id] = (center_point, frame_count)

                if track.is_confirmed() and track.time_since_update == 0:
                    if class_id in arac_siniflari:
                        arac_sayilari[class_id] += 1
                        aktif_araclar[class_id] += 1

                if class_id is not None and class_id in arac_siniflari:
                    text = f"ID: {track_id} - {sinif_isimleri[class_id]}"
                else:
                    text = f"ID: {track_id}"

                cv2.putText(lane_overlay, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.putText(lane_overlay, f"FPS: {fps_calc:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(lane_overlay, f"Kare: {frame_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        if car_offset is not None:
            direction = "sol" if car_offset > 0 else "sağ"
            cv2.putText(lane_overlay, f"Ofset: {abs(car_offset):.2f}m {direction}", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        y_pos = 30
        cv2.putText(lane_overlay, "Anlik Araclar:", (width - 200, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        y_pos += 30

        for class_id in arac_siniflari:
            sinif_text = f"{sinif_isimleri[class_id]}: {aktif_araclar[class_id]}"
            cv2.putText(lane_overlay, sinif_text, (width - 200, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            y_pos += 30

        y_pos = height - 150
        cv2.putText(lane_overlay, "Toplam Araclar:", (width - 200, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        y_pos += 30

        for class_id in arac_siniflari:
            sinif_text = f"{sinif_isimleri[class_id]}: {arac_sayilari[class_id]}"
            cv2.putText(lane_overlay, sinif_text, (width - 200, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            y_pos += 30

            hough_img = np.zeros_like(frame)
            for x1, y1, x2, y2, _ in lines:
                cv2.line(hough_img, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.imwrite(f"output/{frame_count:04d}_5_hough_lines.jpg", hough_img)

            cv2.imwrite(f"output/{frame_count:04d}_6_final_overlay.jpg", lane_overlay)

        out.write(lane_overlay)
        processing_time = time.time() - start_time

    print(f"Toplam islenen kare sayisi: {frame_count}")
    print(f"Tespit edilen arac sayilari:")
    for class_id, count in arac_sayilari.items():
        print(f"  {sinif_isimleri[class_id]}: {count}")

    cap.release()
    out.release()

    return arac_sayilari

In [14]:
output_path = "entegre_sonuc_video.mp4"
print("Video işleniyor...")

arac_sayilari = process_integrated_video(test_video, output_path, save_frames=True)

print("İşlenmiş video indiriliyor...")
files.download(output_path)

print("İşlem tamamlandı!")

Video işleniyor...

0: 384x640 1 potted plant, 58.3ms
Speed: 6.8ms preprocess, 58.3ms inference, 323.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 potted plant, 10.3ms
Speed: 2.0ms preprocess, 10.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 potted plant, 10.4ms
Speed: 2.1ms preprocess, 10.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.4ms
Speed: 2.1ms preprocess, 10.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.5ms
Speed: 2.3ms preprocess, 10.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.2ms
Speed: 2.1ms preprocess, 10.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.1ms
Speed: 2.1ms preprocess, 10.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 motorcycle, 10.1ms
Speed: 2.0ms prepr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

İşlem tamamlandı!
